#Predikce infarktu myokardu s využitím neuronových sítí

Cílem úlohy je na základě několika málo atributů určit, jestli člověk prodělal infarkt myokardu. Data set je převzatý a upravený z repozitáře [Kaggle](https://www.kaggle.com/fedesoriano/stroke-prediction-dataset?select=healthcare-dataset-stroke-data.csv).

Použité atributy:
* pohlaví - 0: muž, 1: žena, 2: nespecifikováno
* věk
* hypertenze -  zvýšený krevní tlak 0: ne, 1: ano
* nemoc_srdce - 0: neléčí se s žádnou srdeční nemocí, 1: trpí srdeční nemocí
* manzelstvi - 0: nikdy nebyl(a) ženatý/vdaná, 1: byl(a) nebo je ženatý/vdaná
* zamestnani - 0: dite, 1: nikdy nepracoval(a), 2: ve verejnem sektoru, 3: v soukromem sektoru, 4: OSVC
* bydliste - 0: venkov, 1: mesto
* cukr - množství cukru v krvi
* bmi - body mass index, 0 pokud není známo
* koureni - 0: neznamé, 1: nikdy nekouřil(a), 2: přestal(a) kouřit, 3: kouří
* infarkt - 0: nikdy neprodělal(a) infarkt, 1: prodělal(a) infarkt

In [ ]:
import pandas as pd

data = pd.read_csv('https://raw.githubusercontent.com/mlcollege/ai-academy/main/07-klasifikace/data/infarkt.csv', index_col='id')
data.head()

,pohlavi,vek,hypertenze,nemoc_srdce,manzelstvi,zamestnani,bydliste,cukr,bmi,koureni,infarkt
id,,,,,,,,,,,
9046,0,67.0,0,1,1,3,1,228.69,36.6,2,1
51676,1,61.0,0,0,1,4,0,202.21,0.0,1,1
31112,0,80.0,0,1,1,3,0,105.92,32.5,1,1
60182,1,49.0,0,0,1,3,1,171.23,34.4,3,1
1665,1,79.0,1,0,1,4,0,174.12,24.0,1,1


## Příprava dat

Nejprve si rozdělíme data na trénovací a testovací část.


In [ ]:
from sklearn.model_selection import train_test_split

vstupni_atributy = ['pohlavi', 'vek',	'hypertenze', 'nemoc_srdce', 'manzelstvi',
                    'zamestnani', 'bydliste', 'cukr', 'bmi', 'koureni']
vystupni_atribut = 'infarkt'

X_train, X_test, y_train, y_test = train_test_split(data[vstupni_atributy], data[vystupni_atribut], test_size=0.3, random_state=4)
print('Velikost trenovaci casti: {}'.format(len(X_train)))
print('Velikost testovaci casti: {}'.format(len(X_test)))

Velikost trenovaci casti: 3577
Velikost testovaci casti: 1533


Pro použití v neuronových sítích je vhodné vstupní data předzpracovat. Experimentujte s různými způsoby předzpracování.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

column_trans = ColumnTransformer(
    [('ohe', OneHotEncoder(categories='auto'),['pohlavi', 'zamestnani', 'koureni']),
     ('std', StandardScaler(), ['vek', 'cukr', 'bmi'])
    ], remainder='passthrough')

column_trans.fit(X_train)

X_train_trans = column_trans.transform(X_train)
X_test_trans = column_trans.transform(X_test)

#Vytvoření a natrénování modelu

Použijeme stejnou architekturu jako v případě regese. Jediný rozdíl bude v použití Sigmoidy jako aktivační funkce poslední vrstvy. Zkuste experimentovat i s jinými architekturami neuronové sítě.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

model = Sequential()

model.add(Dense(50, input_shape=(19, )))
model.add(Activation('tanh'))
model.add(Dense(50))
model.add(Activation('tanh'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

Dále je potřeba model zkompilovat. Pro kompilaci musíme zadat několik důležitých parametrů:
* **loss** - chybová funkce, kterou budeme minimalizovat. Pro binární klasifikaci použijeme *binary_crossentropy*.
* **optimizer** - algoritmů pro hledání minima chybové funkce existuje celá řada. Všechny vycházejí z podobných principů a liší se hlavně ve způsobech, jakými minimalizují šanci uvíznutí v nějakém špatném lokálním extrému. Jedním z nejčastěji používaných optimalizátorů je [Adam](https://keras.io/api/optimizers/adam/).
* **metrics** - během trénování umožňuje algoritmus průběžné vypisování aktuální kvality modelu. Pokud o to stojíme, je třeba specifikovat konkrétní metriku. V našem případě jsme zvolili _accuracy_.

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Samotné trénování se provádí metodou fit a je třeba opět specifikovat několik parametrů:

* trénovací data - X_train_std a y_train
* **batch_size** - počet trénovacích příkladů, které jsou zpracovávány současně.
* **epochs** - počet epoch (průchodů celými trénovacími daty).
* **verbose** - úroveň množství výpisů během trénování
* **validation_data** - data, použitá pro validaci

In [ ]:
model.fit(X_train_trans, y_train,
          batch_size = 64, epochs = 50, verbose=1,
          validation_data=(X_test_trans, y_test))

Epoch 1/50
56/56 [==============================] - 1s 6ms/step - loss: 0.2732 - accuracy: 0.9223 - val_loss: 0.1582 - val_accuracy: 0.9550
Epoch 2/50
56/56 [==============================] - 0s 2ms/step - loss: 0.1716 - accuracy: 0.9486 - val_loss: 0.1493 - val_accuracy: 0.9550
Epoch 3/50
56/56 [==============================] - 0s 2ms/step - loss: 0.1630 - accuracy: 0.9494 - val_loss: 0.1482 - val_accuracy: 0.9550
Epoch 4/50
56/56 [==============================] - 0s 2ms/step - loss: 0.1588 - accuracy: 0.9486 - val_loss: 0.1474 - val_accuracy: 0.9563
Epoch 5/50
56/56 [==============================] - 0s 3ms/step - loss: 0.1579 - accuracy: 0.9491 - val_loss: 0.1471 - val_accuracy: 0.9556
Epoch 6/50
56/56 [==============================] - 0s 3ms/step - loss: 0.1574 - accuracy: 0.9491 - val_loss: 0.1463 - val_accuracy: 0.9556
Epoch 7/50
56/56 [==============================] - 0s 2ms/step - loss: 0.1567 - accuracy: 0.9491 - val_loss: 0.1469 - val_accuracy: 0.9550
Epoch 8/50
56/56 [==

Výstupní hodnoty je před evaluací třeba převést z pravděpodobností na hodnoty 0 a 1.

In [ ]:
from numpy import int32
y_pred = model.predict(X_test_trans)
y_pred = (y_pred >= 0.2).astype(int32)

Pro vyhodnocení modelu je vhodné se dívat na precision, recall a f1-score. Accuracy bude díky nevyváženosti data setu vysoká.

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score

print ("Celkova spravnost klasifikace pro testovaci data: {:.2f}".format(accuracy_score(y_test, y_pred)))
print ()
print(metrics.classification_report(y_test, y_pred))

Celkova spravnost klasifikace pro testovaci data: 0.93

              precision    recall  f1-score   support

           0       0.97      0.96      0.96      1465
           1       0.26      0.28      0.27        68

    accuracy                           0.93      1533
   macro avg       0.61      0.62      0.62      1533
weighted avg       0.93      0.93      0.93      1533

